<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/Simulationofanoilstabilizationprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simulation of an oil stabilization process
In this notebook we will establish code for using NeqSim for simulation of an oil stabilization process

In [1]:
# Setting up the Colab page to use neqsim
%%capture
!pip install neqsim==3.0.12
import neqsim
from neqsim.thermo.thermoTools import *

# Process flow diagram

In [2]:
inputdata = {
  'feedFlowRateWell': 10,
  'wellPressure': 180.0,
  'wellTemperature':100.0,
  'topsidePressure': 90.0,
  'topsideTemperature': 5.0,
  'firstStagePressure': 75.0,
  'temperatureOilHeater' : 75.9,
  'secondStagePressure': 8.6,
  'thirdStagePressure': 1.9,
  'firstStageSuctionCoolerTemperature': 25.3,
  'secondStageSuctionCoolerTemperature': 24.5,
  'thirdStageSuctionCoolerTemperature':25.0,
  'firstStageExportCoolerTemperature': 25.3,
  'secondStageExportCoolerTemperature': 24.5
}

In [6]:
from neqsim.thermo import fluid, printFrame
from neqsim.process import compressor, cooler, separator3phase, getProcess, clearProcess, mixer, heater, stream, pump, separator, runProcess, stream, saturator, valve, filters, heatExchanger, simpleTEGAbsorber,distillationColumn, waterStripperColumn, recycle, setpoint, calculator

clearProcess()

wellFluid = fluid('pr')
wellFluid.addComponent("nitrogen", 0.08)
wellFluid.addComponent("CO2", 3.56)
wellFluid.addComponent("methane", 87.36)
wellFluid.addComponent("ethane", 4.02)
wellFluid.addComponent("propane", 1.54)
wellFluid.addComponent("i-butane", 0.2)
wellFluid.addComponent("n-butane", 0.42)
wellFluid.addComponent("i-pentane", 0.15)
wellFluid.addComponent("n-pentane", 0.20)

wellFluid.addTBPfraction("C6", 3.24, 84.99/1000.0, 695.0/1000.0)
wellFluid.addTBPfraction("C7", 1.34, 97.87/1000.0, 718.0/1000.0)
wellFluid.addTBPfraction("C8", 1.33, 111.54/1000.0, 729.0/1000.0)
wellFluid.addTBPfraction("C9", 1.19, 126.1/1000.0, 749.0/1000.0)
wellFluid.addTBPfraction("C10", 1.15, 140.14/1000.0, 760.0/1000.0)
wellFluid.addTBPfraction("C11", 1.69, 175.0/1000.0, 830.0/1000.0)
wellFluid.addTBPfraction("C12", 1.5, 280.0/1000.0, 914.0/1000.0)
wellFluid.addTBPfraction("C13", 2.103, 560.0/1000.0, 980.0/1000.0)

wellFluid.setMixingRule(2)
wellFluid.init(0)

wellFluid.setMolarComposition([0.08, 3.56, 87.36, 4.02, 1.54, 0.2, 0.42, 0.15, 0.2, 1.24, 1.34, 1.33, 1.19, 1.15, 1.69, 1.5, 1.03])

wellStream = stream("dry well stream", wellFluid)
wellStream.setFlowRate(inputdata['feedFlowRateWell'], "MSm3/day")
wellStream.setTemperature(inputdata['wellTemperature'], "C")
wellStream.setPressure(inputdata['wellPressure'], "bara")

saturatedFeedGas = saturator("water saturator", wellStream)

waterSaturatedFeedGas = stream("water saturated feed gas", saturatedFeedGas.getOutStream())

feedTPsetter = heater('inletTP', waterSaturatedFeedGas)
feedTPsetter.setOutPressure(inputdata['topsidePressure'], "bara")
feedTPsetter.setOutTemperature(inputdata['topsideTemperature'], "C")

chokeValve = valve('valve 1', feedTPsetter.getOutStream())
chokeValve.setOutletPressure(inputdata['firstStagePressure'], 'bara')

feedToOffshoreProcess = stream("feed to offshore", chokeValve.getOutStream())

firstStageSeparator = separator3phase("1st stage separator", feedToOffshoreProcess)

oilHeaterFromFirstStage = heater("oil heater second stage", firstStageSeparator.getOilOutStream())
oilHeaterFromFirstStage.setOutTemperature(inputdata['temperatureOilHeater'],'C')

oilThrotValve = valve("valve oil from first stage", oilHeaterFromFirstStage.getOutStream())
oilThrotValve.setOutletPressure(inputdata['secondStagePressure'])

secondStageSeparator = separator3phase("2nd stage separator", oilThrotValve.getOutStream())

oilThrotValve2 = valve("valve oil from second stage", secondStageSeparator.getOilOutStream())
oilThrotValve2.setOutletPressure(inputdata['thirdStagePressure'])

thirdStageSeparator = separator3phase("3rd stage separator", oilThrotValve2.getOutStream())

oilThirdStageToSep =  wellStream.clone()
oilThirdStageToSep.setName("resyc oil")
thirdStageSeparator.addStream(oilThirdStageToSep)

stableOil = stream("stable oil", thirdStageSeparator.getOilOutStream())

stableOilPump = pump("stable oil pump", stableOil,p=15.0)

firstStageCooler = cooler("1st stage cooler", thirdStageSeparator.getGasOutStream())
firstStageCooler.setOutTemperature(inputdata['firstStageSuctionCoolerTemperature'],'C')

firstStageScrubber = separator("1st stage scrubber", firstStageCooler.getOutStream())

firstStageCompressor = compressor("1st stage compressor", firstStageScrubber.getGasOutStream())
firstStageCompressor.setOutletPressure(inputdata['secondStagePressure'])
firstStageCompressor.setIsentropicEfficiency(0.75)

secondStageCooler = cooler("2nd stage cooler", firstStageCompressor.getOutStream())
secondStageCooler.setOutTemperature(inputdata['secondStageSuctionCoolerTemperature'],'C')

secondStageScrubber = separator("2nd stage scrubber", secondStageCooler.getOutStream())

secondStageCompressor = compressor("2nd stage compressor", secondStageScrubber.getGasOutStream())
secondStageCompressor.setOutletPressure(inputdata['firstStagePressure'])
secondStageCompressor.setIsentropicEfficiency(0.75)

richGasMixer = mixer("fourth Stage mixer")
richGasMixer.addStream(secondStageCompressor.getOutStream())
richGasMixer.addStream(firstStageSeparator.getGasOutStream())

dewPointControlCooler = cooler("dew point cooler", richGasMixer.getOutStream())
dewPointControlCooler.setOutTemperature(inputdata['thirdStageSuctionCoolerTemperature'],'C')

dewPointScrubber = separator("dew point scrubber", dewPointControlCooler.getOutStream())

lpLiqmixer = mixer("LP liq gas mixer");
lpLiqmixer.addStream(firstStageScrubber.getLiquidOutStream());
lpLiqmixer.addStream(secondStageScrubber.getLiquidOutStream());
lpLiqmixer.addStream(dewPointScrubber.getLiquidOutStream());

lpResycle = recycle("LP liq resycle")
lpResycle.addStream(lpLiqmixer.getOutStream())
lpResycle.setOutletStream(oilThirdStageToSep)

exportCompressor1 = compressor("export 1st stage", dewPointScrubber.getGasOutStream())
exportCompressor1.setOutletPressure(140.0)
exportCompressor1.setIsentropicEfficiency(0.75)

exportInterstageCooler = cooler("interstage stage cooler", exportCompressor1.getOutStream())
exportInterstageCooler.setOutTemperature(inputdata['firstStageExportCoolerTemperature'],'C')

exportCompressor2= compressor("export 2nd stage", exportInterstageCooler.getOutStream())
exportCompressor2.setOutletPressure(200.0)
exportCompressor2.setIsentropicEfficiency(0.75)

exportCooler = cooler("export cooler", exportCompressor1.getOutStream())
exportCooler.setOutTemperature(inputdata['secondStageExportCoolerTemperature'],'C')

exportGas = stream("export gas", exportCooler.getOutStream())

oilprocess = getProcess()


# Start simulation as a thread

In [7]:
#oilprocess.run() #run this to finish thread before continuing
thread = oilprocess.runAsThread()
thread.join(50000) # set maximum time to 50000 msec before continuing
if thread.isAlive():
  thread.interrupt()
  thread.join()

# Read results from simulation

In [8]:
TVP = stableOil.TVP(20.0, 'C')
cricondenbar = exportGas.CCB('bara')
powerComp1 = oilprocess.getUnit("1st stage compressor").getPower()/1.0e3
powerComp2 = oilprocess.getUnit("2nd stage compressor").getPower()/1.0e3
powerExpComp1 = oilprocess.getUnit("export 1st stage").getPower()/1.0e3
powerExpComp2 = oilprocess.getUnit("export 2nd stage").getPower()/1.0e3

print('TVP [bara] ', TVP)
print('cricondenbar [bara] ', cricondenbar)
print('recomp power 1 [kW] ', powerComp1)
print('recomp power 2 [kW] ', powerComp2)
print('exp power 1 [kW] ', powerExpComp1)
print('exp power 2 [kW] ', powerExpComp2)


TVP [bara]  0.9994563400733689
cricondenbar [bara]  88.4576902001664
recomp power 1 [kW]  143.26369311314653
recomp power 2 [kW]  184.03248828270947
exp power 1 [kW]  7584.58555297371
exp power 2 [kW]  3922.0111306267986


In [9]:
from neqsim.thermo import fluid, printFrame
printFrame(waterSaturatedFeedGas.getFluid())

| 0                    | 1          | 2          | 3           | 4   | 5   | 6               |
|:---------------------|:-----------|:-----------|:------------|:----|:----|:----------------|
|                      | total      | GAS        | OIL         |     |     |                 |
| nitrogen             | 7.33585E-4 | 8.31521E-4 | 2.03897E-4  |     |     | [mole fraction] |
| CO2                  | 3.26445E-2 | 3.4396E-2  | 2.31715E-2  |     |     | [mole fraction] |
| methane              | 8.01074E-1 | 8.7639E-1  | 3.93731E-1  |     |     | [mole fraction] |
| ethane               | 3.68626E-2 | 3.72901E-2 | 3.45504E-2  |     |     | [mole fraction] |
| propane              | 1.41215E-2 | 1.30659E-2 | 1.98308E-2  |     |     | [mole fraction] |
| i-butane             | 1.83396E-3 | 1.56775E-3 | 3.27374E-3  |     |     | [mole fraction] |
| n-butane             | 3.85132E-3 | 3.13332E-3 | 7.73461E-3  |     |     | [mole fraction] |
| i-pentane            | 1.37547E-3 | 9.94719E-4 |

# Save process to file

In [10]:
oilprocess.save('oilpro.neqsim')

# Sizing of heat exchangers

https://www.engineeringtoolbox.com/heat-transfer-coefficients-exchangers-d_450.html




In [11]:
import math
U_Wm2K = 100.0

water_inlet_temperature = 15.0
water_outlet_temperature = oilprocess.getUnit('1st stage cooler').getInStream().getTemperature("C")-10.0
gas_inlet_temperature=oilprocess.getUnit('1st stage cooler').getInStream().getTemperature("C")
gas_outlet_temperature = oilprocess.getUnit('1st stage cooler').getOutStream().getTemperature("C")


deltaTA = gas_inlet_temperature-water_outlet_temperature
deltaTB = gas_outlet_temperature-water_inlet_temperature

LMTD = (deltaTA-deltaTB)/(math.log(deltaTA/deltaTB))

Q = -oilprocess.getUnit('1st stage cooler').getEnergyInput()
A = Q/U_Wm2K/LMTD
print("LMTD ", round(LMTD,3), " degC")
print("heat exchanger duty ", round(oilprocess.getUnit('1st stage cooler').getEnergyInput()/1e6,3), " MW")
print("heat exchanger area ", round(A,3), " m^2")

cPwater = 4200.0 #J/kgC
densityWater = 1000.0 #kg/m3
waterCirculationRate =  Q/(cPwater*(water_outlet_temperature-water_inlet_temperature))*3600/densityWater
print("water circulation rate ", round(waterCirculationRate,3), " m3/hr")


water_inlet_temperature = 15.0
water_outlet_temperature = oilprocess.getUnit('2nd stage cooler').getInStream().getTemperature("C")-10.0
gas_inlet_temperature=oilprocess.getUnit('2nd stage cooler').getInStream().getTemperature("C")
gas_outlet_temperature = oilprocess.getUnit('2nd stage cooler').getOutStream().getTemperature("C")


deltaTA = gas_inlet_temperature-water_outlet_temperature
deltaTB = gas_outlet_temperature-water_inlet_temperature

LMTD = (deltaTA-deltaTB)/(math.log(deltaTA/deltaTB))
Q = -oilprocess.getUnit('2nd stage cooler').getEnergyInput()
A = Q/U_Wm2K/LMTD
print("LMTD2 ", round(LMTD,3), " degC")
print("heat exchanger duty2 ", round(oilprocess.getUnit('2nd stage cooler').getEnergyInput()/1e6,3), " MW")
print("heat exchanger area2 ", round(A,3), " m^2")

cPwater = 4200.0 #J/kgC
densityWater = 1000.0 #kg/m3
waterCirculationRate =  Q/(cPwater*(water_outlet_temperature-water_inlet_temperature))*3600/densityWater
print("water circulation rate ", round(waterCirculationRate,3), " m3/hr")

water_inlet_temperature = 15.0
water_outlet_temperature = oilprocess.getUnit('interstage stage cooler').getInStream().getTemperature("C")-10.0
gas_inlet_temperature=oilprocess.getUnit('interstage stage cooler').getInStream().getTemperature("C")
gas_outlet_temperature = oilprocess.getUnit('interstage stage cooler').getOutStream().getTemperature("C")


deltaTA = gas_inlet_temperature-water_outlet_temperature
deltaTB = gas_outlet_temperature-water_inlet_temperature

LMTD = (deltaTA-deltaTB)/(math.log(deltaTA/deltaTB))
Q = -oilprocess.getUnit('interstage stage cooler').getEnergyInput()
A = Q/U_Wm2K/LMTD
print("LMTD3 ", round(LMTD,3), " degC")
print("heat exchanger duty3 ", round(oilprocess.getUnit('interstage stage cooler').getEnergyInput()/1e6,3), " MW")
print("heat exchanger area3 ", round(A,3), " m^2")

cPwater = 4200.0 #J/kgC
densityWater = 1000.0 #kg/m3
waterCirculationRate =  Q/(cPwater*(water_outlet_temperature-water_inlet_temperature))*3600/densityWater
print("water circulation rate ", round(waterCirculationRate,3), " m3/hr")

water_inlet_temperature = 15.0
water_outlet_temperature = oilprocess.getUnit('export cooler').getInStream().getTemperature("C")-10.0
gas_inlet_temperature=oilprocess.getUnit('export cooler').getInStream().getTemperature("C")
gas_outlet_temperature = oilprocess.getUnit('export cooler').getOutStream().getTemperature("C")


deltaTA = gas_inlet_temperature-water_outlet_temperature
deltaTB = gas_outlet_temperature-water_inlet_temperature

LMTD = (deltaTA-deltaTB)/(math.log(deltaTA/deltaTB))
Q = -oilprocess.getUnit('export cooler').getEnergyInput()
A = Q/U_Wm2K/LMTD
print("LMTD4 ", round(LMTD,3), " degC")
print("heat exchanger duty4 ", round(oilprocess.getUnit('export cooler').getEnergyInput()/1e6,3), " MW")
print("heat exchanger area4 ", round(A,3), " m^2")

cPwater = 4200.0 #J/kgC
densityWater = 1000.0 #kg/m3
waterCirculationRate =  Q/(cPwater*(water_outlet_temperature-water_inlet_temperature))*3600/densityWater
print("water circulation rate ", round(waterCirculationRate,3), " m3/hr")

LMTD  10.149  degC
heat exchanger duty  -0.144  MW
heat exchanger area  142.009  m^2
water circulation rate  2.881  m3/hr
LMTD2  9.748  degC
heat exchanger duty2  -0.213  MW
heat exchanger area2  218.769  m^2
water circulation rate  2.053  m3/hr
LMTD3  10.149  degC
heat exchanger duty3  -12.344  MW
heat exchanger area3  12162.057  m^2
water circulation rate  183.142  m3/hr
LMTD4  9.748  degC
heat exchanger duty4  -12.53  MW
heat exchanger area4  12854.161  m^2
water circulation rate  185.908  m3/hr


# Sizing of gas scrubbers


In [12]:
gas_load_factor = 0.1
gas_density = oilprocess.getUnit('1st stage scrubber').getGasOutStream().getFluid().getDensity("kg/m3")
oil_density = oilprocess.getUnit('1st stage scrubber').getLiquidOutStream().getFluid().getDensity("kg/m3")
flowRate = oilprocess.getUnit('1st stage scrubber').getGasOutStream().getFluid().getFlowRate("m3/sec")

V_t = gas_load_factor*math.sqrt((oil_density-gas_density)/gas_density)
A_scrub = flowRate/V_t

D_scrub = math.sqrt(A_scrub/(3.14/4.0))

print("gas density ", round(gas_density,3), " [kg/m3)]")
print("oil density ", round(oil_density,3), " [kg/m3]")
print("Maximum gas velocity ", round(V_t,3), " [m/sec]")
print("Minimum scrubber internal diameter ", round(D_scrub,3), " [m]")

gas_load_factor = 0.1
gas_density = oilprocess.getUnit('2nd stage scrubber').getGasOutStream().getFluid().getDensity("kg/m3")
oil_density = oilprocess.getUnit('2nd stage scrubber').getLiquidOutStream().getFluid().getDensity("kg/m3")
flowRate = oilprocess.getUnit('2nd stage scrubber').getGasOutStream().getFluid().getFlowRate("m3/sec")

V_t = gas_load_factor*math.sqrt((oil_density-gas_density)/gas_density)
A_scrub = flowRate/V_t

D_scrub = math.sqrt(A_scrub/(3.14/4.0))

print("gas density ", round(gas_density,3), " [kg/m3)]")
print("oil density ", round(oil_density,3), " [kg/m3]")
print("Maximum gas velocity ", round(V_t,3), " [m/sec]")
print("Minimum scrubber internal diameter ", round(D_scrub,3), " [m]")

gas_load_factor = 0.1
gas_density = oilprocess.getUnit('dew point scrubber').getGasOutStream().getFluid().getDensity("kg/m3")
oil_density = oilprocess.getUnit('dew point scrubber').getLiquidOutStream().getFluid().getDensity("kg/m3")
if(abs(oil_density-gas_density)<10.0):
  oil_density = 800.0
flowRate = oilprocess.getUnit('dew point scrubber').getGasOutStream().getFluid().getFlowRate("m3/sec")

V_t = gas_load_factor*math.sqrt((oil_density-gas_density)/gas_density)
A_scrub = flowRate/V_t

D_scrub = math.sqrt(A_scrub/(3.14/4.0))

print("gas density ", round(gas_density,3), " [kg/m3)]")
print("oil density ", round(oil_density,3), " [kg/m3]")
print("Maximum gas velocity ", round(V_t,3), " [m/sec]")
print("Minimum scrubber internal diameter ", round(D_scrub,3), " [m]")

gas density  2.641  [kg/m3)]
oil density  691.885  [kg/m3]
Maximum gas velocity  1.616  [m/sec]
Minimum scrubber internal diameter  0.518  [m]
gas density  11.256  [kg/m3)]
oil density  620.406  [kg/m3]
Maximum gas velocity  0.736  [m/sec]
Minimum scrubber internal diameter  0.335  [m]
gas density  64.152  [kg/m3)]
oil density  440.27  [kg/m3]
Maximum gas velocity  0.242  [m/sec]
Minimum scrubber internal diameter  2.432  [m]


In [13]:
printFrame(oilprocess.getUnit('dew point scrubber').getFluid())

| 0                    | 1           | 2           | 3   | 4   | 5   | 6               |
|:---------------------|:------------|:------------|:----|:----|:----|:----------------|
|                      | total       | GAS         |     |     |     |                 |
| nitrogen             | 8.81942E-4  | 8.81942E-4  |     |     |     | [mole fraction] |
| CO2                  | 3.38677E-2  | 3.38677E-2  |     |     |     | [mole fraction] |
| methane              | 9.16808E-1  | 9.16808E-1  |     |     |     | [mole fraction] |
| ethane               | 3.44599E-2  | 3.44599E-2  |     |     |     | [mole fraction] |
| propane              | 9.5965E-3   | 9.5965E-3   |     |     |     | [mole fraction] |
| i-butane             | 8.87449E-4  | 8.87449E-4  |     |     |     | [mole fraction] |
| n-butane             | 1.48953E-3  | 1.48953E-3  |     |     |     | [mole fraction] |
| i-pentane            | 2.95602E-4  | 2.95602E-4  |     |     |     | [mole fraction] |
| n-pentane          